### Implementing custom sequences
- we need to be able to get the length of a sequence
- we need to be able to get element by index of it
Thanks to this we should be able to iteratate through elements and get element by index using []

`__len__`
`__getitem__`

In [1]:
my_list = [1,2,3,4,5,6]
my_list.__getitem__(2), my_list.__len__()

(3, 6)

In [2]:
my_list.__getitem__(slice(None, None, -1))

[6, 5, 4, 3, 2, 1]

In [3]:
index_ = 0
while True:
    try:
        print(my_list.__getitem__(index_))
    except IndexError:
        break
    index_ += 1



1
2
3
4
5
6


In [4]:
class Silly:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        print("Called __len__")
        return "this is silly"

    def __getitem__(self, value: int):
        print(f"Called __getitem__ with index at {value}")
        return "This is a silly element"

In [5]:
s = Silly(10)

In [6]:
try:
    len(s)
except TypeError as e:
    print(e)
    

Called __len__
'str' object cannot be interpreted as an integer


In [7]:
class Silly:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        print("Called __len__")
        return self.n

    def __getitem__(self, value: int):
        print(f"print: >> Called __getitem__ with index at {value}")
        if isinstance(value, int) and (value < 0 or value >= self.n):
            print("print: >> IndexError")
            raise IndexError
        return "This is a silly element"

In [8]:
s = Silly(5)

In [9]:
s.__getitem__(4)

print: >> Called __getitem__ with index at 4


'This is a silly element'

In [10]:
# for loop passes expected index value automatically!
for item in s:
    print(item)

print: >> Called __getitem__ with index at 0
This is a silly element
print: >> Called __getitem__ with index at 1
This is a silly element
print: >> Called __getitem__ with index at 2
This is a silly element
print: >> Called __getitem__ with index at 3
This is a silly element
print: >> Called __getitem__ with index at 4
This is a silly element
print: >> Called __getitem__ with index at 5
print: >> IndexError


In [11]:
s[1:3]  # slice is passed as a value

print: >> Called __getitem__ with index at slice(1, 3, None)


'This is a silly element'

In [12]:
from functools import lru_cache


class Fib:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        return self.n

    def __getitem__(self, s: int | slice):
        if isinstance(s, int):
            if s < 0 or s >= self.n:
                raise IndexError
            
            return self._fib(s)

    @lru_cache((2 ** 31) - 1)
    def _fib(self, n):
        if n < 2:
            return 1
        return self._fib(n - 1) + self._fib(n - 2)
            

In [13]:
f = Fib(8)

In [14]:
f[0], f[3]

(1, 3)

In [15]:
list(f)

[1, 1, 2, 3, 5, 8, 13, 21]

In [16]:
for item in f:
    print(item)

1
1
2
3
5
8
13
21


In [17]:
from functools import lru_cache


class Fib:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        return self.n

    def __getitem__(self, s: int | slice) -> int | list:
        if isinstance(s, int):
            if s < 0:
                # handle negative indexes
                s = self.n + s

            if s < 0 or s >= self.n:
                raise IndexError
            
            return self._fib(s)

        # handle slices (or any object that implements slice interface)
        start_inx, end_inx, step_size = s.indices(self.n)
        return [self._fib(i) for i in range(start_inx, end_inx, step_size)]

    @lru_cache((2 ** 31) - 1)
    def _fib(self, n):
        if n < 2:
            return 1
        return self._fib(n - 1) + self._fib(n - 2)

In [18]:
f = Fib(10)

In [19]:
print(f[-1], list(f))

55 [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


In [20]:
f[2:7]

[2, 3, 5, 8, 13]

In [21]:
f[5:100]

[8, 13, 21, 34, 55]

In [22]:
f[::-1]

[55, 34, 21, 13, 8, 5, 3, 2, 1, 1]

### Assignments in Mutable Sequences
This will only work for mutable sequence types!

In [23]:
l = [1,2,3,4,5]
id(l), l[0:3]


(4473316416, [1, 2, 3])

In [24]:
l[0:3] = (6,7,8,9)
id(l), l

(4473316416, [6, 7, 8, 9, 4, 5])

In [25]:
l =  [1,2,3,4,5,6]
id(l)

4472825216

In [26]:
l[2:5] = ""  # empty iterable will remove elements from the list
id(l), l

(4472825216, [1, 2, 6])

In [27]:
l[0:0] = "python"  # neat trick to insert elements into the list
id(l), l

(4472825216, ['p', 'y', 't', 'h', 'o', 'n', 1, 2, 6])

In [28]:
l[2:2] = ("a", 100, 200)
id(l), l

(4472825216, ['p', 'y', 'a', 100, 200, 't', 'h', 'o', 'n', 1, 2, 6])

In [29]:
l = [1,2,3,4,5,6,7]
id(l), l

(4473210048, [1, 2, 3, 4, 5, 6, 7])

In [30]:
l[0:3] = {100, "x", "a"}
id(l), l

(4473210048, ['x', 100, 'a', 4, 5, 6, 7])

### Extended slices - length of the slice and length of replaced slice must be the same

In [31]:
l = [1,2,3,4,5,6,7]
id(l), l

(4473396736, [1, 2, 3, 4, 5, 6, 7])

In [32]:
l[0:5:2]

[1, 3, 5]

In [33]:
l[0:5:2] = "abc"
id(l), l

(4473396736, ['a', 2, 'b', 4, 'c', 6, 7])

In [34]:
try:
    l[0:5:2] = "python"
except ValueError as e:
    print(e)

attempt to assign sequence of size 6 to extended slice of size 3


### Implementing Custom Sequences, Part 2

In [35]:
class MyClass:

    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"<{type(self).__name__}, name={self.name}>"

    def __add__(self, other):
        print(f"__add__ called with {other}")
        return "hello from __add__"

    def __iadd__(self, other):
        print(f"__iadd__ called with {other}")
        return "hello from __iadd__"


c1 = MyClass("c1")
c2 = MyClass("c2")

In [36]:
c1 + c2

__add__ called with <MyClass, name=c2>


'hello from __add__'

In [37]:
c1 += c2

__iadd__ called with <MyClass, name=c2>


In [38]:
c1, c2

('hello from __iadd__', <MyClass, name=c2>)

In [39]:
class MyClass:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"<{type(self).__name__}, name={self.name}>"

    def __add__(self, other: MyClass) -> MyClass:
        return MyClass(self.name + str(other.name))

    def __iadd__(self, other: MyClass) -> MyClass:
        self.name += str(other.name)
        return self

In [40]:
c1 = MyClass("c1")
c2 = MyClass("c2")

In [41]:
c3 = c1 + c2
print(id(c3), c3)

4473491040 <MyClass, name=c1c2>


In [42]:
print(id(c1))
c1 += c2
print(id(c1), c1)

4473488496
4473488496 <MyClass, name=c1c2>


In [43]:
class MyClass:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"<{type(self).__name__}, name={self.name}>"

    def __add__(self, other: MyClass) -> MyClass:
        return MyClass(self.name + str(other.name))

    def __iadd__(self, other: MyClass) -> MyClass:
        self.name += str(other.name)
        return self

    def __mul__(self, other: int) -> MyClass:
        return MyClass(self.name * other)

    def __imul(self, other: int) -> MyClass:
        self.name *= other
        return self

In [44]:
c1 = MyClass("python")

In [45]:
c1 *  5

<MyClass, name=pythonpythonpythonpythonpython>

In [46]:
c1 *= 2
c1

<MyClass, name=pythonpython>

In [47]:
c1 = MyClass("abcd")

In [48]:
try:
    3 * c1  # __rmul__ not implemented
except TypeError as e:
    print(e)

unsupported operand type(s) for *: 'int' and 'MyClass'


In [49]:
class MyClass:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"<{type(self).__name__}, name={self.name}>"

    def __add__(self, other: MyClass) -> MyClass:
        return MyClass(self.name + str(other.name))

    def __iadd__(self, other: MyClass) -> MyClass:
        self.name += str(other.name)
        return self

    def __mul__(self, other: int) -> MyClass:
        return MyClass(self.name * other)

    def __rmul__(self, other: int) -> MyClass:
        return self.__mul__(other)

    def __imul(self, other: int) -> MyClass:
        self.name *= other
        return self

In [50]:
c1 =  MyClass("abcd")
3 * c1

<MyClass, name=abcdabcdabcd>

### `in` operator

In [51]:
class MyClass:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"<{type(self).__name__}, name={self.name}>"

    def __add__(self, other: MyClass) -> MyClass:
        return MyClass(self.name + str(other.name))

    def __iadd__(self, other: MyClass) -> MyClass:
        self.name += str(other.name)
        return self

    def __mul__(self, other: int) -> MyClass:
        return MyClass(self.name * other)

    def __rmul__(self, other: int) -> MyClass:
        return self.__mul__(other)

    def __imul(self, other: int) -> MyClass:
        self.name *= other
        return self

    def __contains__(self, value):
        return value in self.name

In [52]:
c1 = MyClass("abcd")
"a" in c1

True

In [53]:
try:
    1 in c1  # let python handle the exception
except TypeError as e:
    print(e)

'in <string>' requires string as left operand, not int


### Custom class for polygon implementation

In [54]:
import numbers


class Point:
    def __init__(self, x, y):
        if isinstance(x, numbers.Real) and isinstance(y, numbers.Real):
            self.x = x
            self.y = y
            self._pt = (x, y)
        else:
            raise TypeError("Point coordinates must be real numbers")

    def __repr__(self):
        return f"{type(self).__name__}(x={self.x}, y={self.y})"

    def __len__(self):
        return len(self._pt)

    def __getitem__(self, s: int | slice):
        return self._pt[s]


In [55]:
p1 = Point(10, 2)

In [56]:
x, y = p1   # thanks to __getitem__ it's possible to unpack
x, y

(10, 2)

In [57]:
p2 = Point(*p1)

In [58]:
p1, p2

(Point(x=10, y=2), Point(x=10, y=2))

In [59]:
class Polygon:
    def __init__(self, *pts):
        self._pts = []
        if pts:
            self._pts = [Point(*p) for p in pts]

    def __repr__(self):
        # make it so it's possible to copy str __repr__ and create valid objects!
        points_str = ",".join([str(p) for p in self._pts])
        return f"{type(self).__name__}({points_str})"

    def __len__(self):
        return len(self._pts)


In [60]:
p = Polygon((0,0), p1, p2, (-5, -2))
p

Polygon(Point(x=0, y=0),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2))

In [61]:
x = Polygon(Point(x=0, y=0),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2))

In [62]:
x

Polygon(Point(x=0, y=0),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2))

In [63]:
class Polygon:
    def __init__(self, *pts):
        self._pts = []
        if pts:
            self._pts = [Point(*p) for p in pts]

    def __repr__(self):
        # make it so it's possible to copy str __repr__ and create valid objects!
        points_str = ",".join([str(p) for p in self._pts])
        return f"{type(self).__name__}({points_str})"

    def __len__(self):
        return len(self._pts)

    def __getitem__(self, s: int | slice):
        return self._pts[s]

In [64]:
p = Polygon(Point(x=0, y=0),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2))

In [65]:
list(p)

[Point(x=0, y=0), Point(x=10, y=2), Point(x=10, y=2), Point(x=-5, y=-2)]

In [66]:
p[::-1]

[Point(x=-5, y=-2), Point(x=10, y=2), Point(x=10, y=2), Point(x=0, y=0)]

In [67]:
class Polygon:
    def __init__(self, *pts):
        self._pts = []
        if pts:
            self._pts = [Point(*p) for p in pts]

    def __repr__(self):
        # make it so it's possible to copy str __repr__ and create valid objects!
        points_str = ",".join([str(p) for p in self._pts])
        return f"{type(self).__name__}({points_str})"

    def __len__(self):
        return len(self._pts)

    def __getitem__(self, s: int | slice):
        return self._pts[s]

    def __add__(self, other: Polygon) -> Polygon:
        if isinstance(other, Polygon):
            new_points = list(self) + list(other)
            return Polygon(*new_points)
        else:
            raise TypeError(f"add operator expects {type(self)} type")
    

In [68]:
p1 = Polygon(Point(x=0, y=0), Point(x=10, y=2), Point(x=10, y=2), Point(x=-5, y=-2))
p2 = Polygon(Point(x=-5, y=-2), Point(x=10, y=2), Point(x=10, y=2), Point(x=0, y=0))

In [69]:
p1, p2

(Polygon(Point(x=0, y=0),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2)),
 Polygon(Point(x=-5, y=-2),Point(x=10, y=2),Point(x=10, y=2),Point(x=0, y=0)))

In [70]:
p1 + p2

Polygon(Point(x=0, y=0),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2),Point(x=-5, y=-2),Point(x=10, y=2),Point(x=10, y=2),Point(x=0, y=0))

In [71]:
try:
    p1 + 10
except TypeError as e:
    print(e)

add operator expects <class '__main__.Polygon'> type


In [72]:
class Polygon:
    def __init__(self, *pts):
        self._pts = []
        if pts:
            self._pts = [Point(*p) for p in pts]

    def __repr__(self):
        # make it so it's possible to copy str __repr__ and create valid objects!
        points_str = ",".join([str(p) for p in self._pts])
        return f"{type(self).__name__}({points_str})"

    def __len__(self):
        return len(self._pts)

    def __getitem__(self, s: int | slice):
        return self._pts[s]

    def __add__(self, other: Polygon) -> Polygon:
        if isinstance(other, Polygon):
            new_points = list(self) + list(other)
            return Polygon(*new_points)
        else:
            raise TypeError(f"add operator expects {type(self)} type")

    def __iadd__(self, other):
        if isinstance(other, Polygon):
            self._pts += list(other)
            return self
        else:
            raise TypeError(f"add operator expects {type(self)} type")


In [73]:
p1 = Polygon(Point(x=0, y=0), Point(x=10, y=2), Point(x=10, y=2), Point(x=-5, y=-2))
p2 = Polygon(Point(x=-5, y=-2), Point(x=10, y=2), Point(x=10, y=2), Point(x=0, y=0))
print(id(p1))
p1 += p2
print(id(p1))

4472944400
4472944400


In [74]:
p1

Polygon(Point(x=0, y=0),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2),Point(x=-5, y=-2),Point(x=10, y=2),Point(x=10, y=2),Point(x=0, y=0))

In [75]:
class Polygon:
    def __init__(self, *pts):
        self._pts = []
        if pts:
            self._pts = [Point(*p) for p in pts]

    def __repr__(self):
        # make it so it's possible to copy str __repr__ and create valid objects!
        points_str = ",".join([str(p) for p in self._pts])
        return f"{type(self).__name__}({points_str})"

    def __len__(self):
        return len(self._pts)

    def __getitem__(self, s: int | slice):
        return self._pts[s]

    def __add__(self, other: Polygon) -> Polygon:
        if isinstance(other, Polygon):
            new_points = list(self) + list(other)
            return Polygon(*new_points)
        else:
            raise TypeError(f"add operator expects {type(self)} type")

    def __iadd__(self, other):
        if isinstance(other, Polygon):
            self._pts += list(other)
            return self
        else:
            raise TypeError(f"add operator expects {type(self)} type")

    def __contains__(self, pt):
        return Point(*pt) in self._pts

    def append(self, pt):
        self._pts.append(Point(*pt))

    def insert(self, i, pt):
        self._pts.insert(i, Point(*pt))

    def extend(self, pts):
        if isinstance(pts, Polygon):
            self._pts += list(pts)
        else:
            self._pts += [Point(*p) for p in pts]


In [76]:
p1 = Polygon(Point(x=0, y=0), Point(x=10, y=2), Point(x=10, y=2), Point(x=-5, y=-2))
p2 = Polygon(Point(x=-5, y=-2), Point(x=10, y=2), Point(x=10, y=2), Point(x=0, y=0))

In [77]:
p1.append((10, 100))

In [78]:
p1

Polygon(Point(x=0, y=0),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2),Point(x=10, y=100))

In [79]:
p1.insert(1, (-100, -100))

In [80]:
p1

Polygon(Point(x=0, y=0),Point(x=-100, y=-100),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2),Point(x=10, y=100))

In [81]:
p1.extend([Point(25, 25), (55, 55)])

In [82]:
p1

Polygon(Point(x=0, y=0),Point(x=-100, y=-100),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2),Point(x=10, y=100),Point(x=25, y=25),Point(x=55, y=55))

In [83]:
p1.extend(p2)

In [84]:
p1

Polygon(Point(x=0, y=0),Point(x=-100, y=-100),Point(x=10, y=2),Point(x=10, y=2),Point(x=-5, y=-2),Point(x=10, y=100),Point(x=25, y=25),Point(x=55, y=55),Point(x=-5, y=-2),Point(x=10, y=2),Point(x=10, y=2),Point(x=0, y=0))

In [85]:
class Polygon:
    def __init__(self, *pts):
        self._pts = []
        if pts:
            self._pts = [Point(*p) for p in pts]

    def __repr__(self):
        # make it so it's possible to copy str __repr__ and create valid objects!
        points_str = ",".join([str(p) for p in self._pts])
        return f"{type(self).__name__}({points_str})"

    def __len__(self):
        return len(self._pts)

    def __getitem__(self, s: int | slice):
        return self._pts[s]

    def __setitem__(self, s: int | slice, value: Point | tuple | list):
        is_single =  False
        try:
            rhs = [Point(*pt) for pt in value]
            is_single = False
        except TypeError:
            try:
                rhs = Point(*value)
                is_single = True
            except TypeError:
                raise TypeError("Invalid Point or iterable of Points")

        if (isinstance(s, int) and is_single) or (isinstance(s, slice) and not is_single):
            self._pts[s] = rhs    
        else:
            raise TypeError("Incompatibile index/slice assignment")

    def __delitem__(self, s: int | slice):
        del self._pts[s]

    def __add__(self, other: Polygon) -> Polygon:
        if isinstance(other, Polygon):
            new_points = list(self) + list(other)
            return Polygon(*new_points)
        else:
            raise TypeError(f"add operator expects {type(self)} type")

    def __iadd__(self, other):
        if isinstance(other, Polygon):
            self._pts += list(other)
            return self
        else:
            raise TypeError(f"add operator expects {type(self)} type")

    def __contains__(self, pt):
        return Point(*pt) in self._pts

    def append(self, pt):
        self._pts.append(Point(*pt))

    def insert(self, i, pt):
        self._pts.insert(i, Point(*pt))

    def extend(self, pts):
        if isinstance(pts, Polygon):
            self._pts += list(pts)
        else:
            self._pts += [Point(*p) for p in pts]

    def pop(self, i):
        return self._pts.pop(i)
